In [66]:
import os
import re
import numpy as np
import pandas as pd

In [67]:
os.chdir("/mnt/lareaulab/reliscu/projects/Chronocell/data")

In [68]:
legend = pd.read_csv("experimental_data_legend.csv")
legend.head()

,File,Type,Species,Cell_type,Method,Experiment_type,Quantity_description,Calculation_method,Notes
0,Eraslan2019_HPA_tableS3.csv,TE,Human,29 tissues,"LC‐MS/MS, RNA-seq",In vivo,Protein‐to‐mRNA ratio,Log-10-transformed ratio of the normalized pro...,NaN
1,Hernandez-Alias2023_PTR_GTEx_tableS2.csv,TE,Human,24 tissues,"TMT MS3, RNA-seq",In vivo,Protein‐to‐mRNA ratio,Log-10-transformed ratio of the normalized pro...,NaN
2,Zheng2025_human_tableS1.csv,TE,Human,>140 cell types,"Ribo-seq, RNA-seq",In vivo; In vitro,Translation efficiency,NaN,"Based on 3,819 ribosomal profiling datasets"
3,Zheng2025_mouse_tableS1.csv,TE,Mouse,>140 cell types,"Ribo-seq, RNA-seq",In vivo; In vitro,Translation efficiency,NaN,"Based on 3,819 ribosomal profiling datasets"
4,Hernandez-Alias2023_TE_Wang2020_tableS2.csv,TE,Human,Brain,"Ribo-seq, RNA-seq",In vivo,Translation efficiency,Median of log-10-transformed Ribo-seq TPM and ...,NaN


In [69]:
mask = legend['Type'].values == "TE"
legend['Path'] = None
legend.loc[mask, 'Path'] = (os.getcwd() + "/translation_rates/" + legend['File'].values)[mask]
legend.loc[~mask, 'Path'] = (os.getcwd() + "/protein_degradation_rates/" + legend['File'].values)[~mask]

In [70]:
mask = (legend['Type'] == "Deg") # & (legend['Species'] == "Human")
legend1 = legend.loc[mask].reset_index(drop=True)

In [6]:
std_columns = ['Study', 'Species', 'Cell_type', 'Method', 'Gene', 'Half-life']

# Format

### Dataset 1

In [73]:
idx = 0
data0 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx])
data0.head()

File                                             Boisvert2011_tableS2.csv
Type                                                                  Deg
Species                                                             Human
Cell_type                                                      HeLa cells
Method                                                  SILAC pulse-chase
Experiment_type                                                  In vitro
Quantity_description                                            Half-life
Calculation_method      Half-lives were calculated using an exponentia...
Notes                          Pulsed for 0.5, 4, 7, 11, 27, and 48 hours
Path                    /mnt/lareaulab/reliscu/projects/Chronocell/dat...
Name: 0, dtype: object


,Protein Accession,Gene,Description,Whole Turnover,Whole Half-Life,Whole Chi2,Cytoplasm Turnover,Cytoplasm Half-Life,Cytoplasm Chi2,Nucleus Turnover,Nucleus Half-Life,Nucleus Chi2,Nucleolus Turnover,Nucleolus Half-Life,Nucleolus Chi2
0,IPI00788723,NaN,similar to hCG1777996,26.045200,NaN,0.006633,NaN,NaN,NaN,NaN,NaN,NaN,24.034901,NaN,NaN
1,IPI00292537,NaN,Isoform 2 of Nuclear factor NF-kappa-B p105 su...,23.990700,NaN,0.043440,0.000000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN,0.000000
2,IPI00018783,ITPA,Inosine triphosphate pyrophosphatase,22.706051,NaN,0.014058,22.577850,-598.668696,0.012268,25.80835,999.000000,0.038288,NaN,NaN,NaN
3,IPI00059279,EXOC4,Exocyst complex component 4,20.688351,NaN,0.006546,16.347550,999.000000,0.016951,24.11875,157.231155,0.000501,13.062000,NaN,NaN
4,IPI00410079,FAM82A2,Isoform 1 of Regulator of microtubule dynamics...,24.819050,NaN,0.008621,23.186899,999.000000,0.013351,24.17310,50.688920,0.000456,27.525101,999.0,0.016516


In [74]:
# Replace negative values with NaN (indicates poor fit model fit)
data0.loc[data0['Cytoplasm Half-Life'] < 0, 'Cytoplasm Half-Life'] = np.nan

# Standardize columns
data0['Half-life'] = data0['Cytoplasm Half-Life']
data0['Cell_type'] = legend1.loc[idx, 'Cell_type']
data0['Method'] = legend1.loc[idx, 'Method']
data0['Species'] = legend1.loc[idx, 'Species']

data0.head()

,Protein Accession,Gene,Description,Whole Turnover,Whole Half-Life,Whole Chi2,Cytoplasm Turnover,Cytoplasm Half-Life,Cytoplasm Chi2,Nucleus Turnover,Nucleus Half-Life,Nucleus Chi2,Nucleolus Turnover,Nucleolus Half-Life,Nucleolus Chi2,Half-life,Cell_type,Method,Species
0,IPI00788723,NaN,similar to hCG1777996,26.045200,NaN,0.006633,NaN,NaN,NaN,NaN,NaN,NaN,24.034901,NaN,NaN,NaN,HeLa cells,SILAC pulse-chase,Human
1,IPI00292537,NaN,Isoform 2 of Nuclear factor NF-kappa-B p105 su...,23.990700,NaN,0.043440,0.000000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN,0.000000,NaN,HeLa cells,SILAC pulse-chase,Human
2,IPI00018783,ITPA,Inosine triphosphate pyrophosphatase,22.706051,NaN,0.014058,22.577850,NaN,0.012268,25.80835,999.000000,0.038288,NaN,NaN,NaN,NaN,HeLa cells,SILAC pulse-chase,Human
3,IPI00059279,EXOC4,Exocyst complex component 4,20.688351,NaN,0.006546,16.347550,999.0,0.016951,24.11875,157.231155,0.000501,13.062000,NaN,NaN,999.0,HeLa cells,SILAC pulse-chase,Human
4,IPI00410079,FAM82A2,Isoform 1 of Regulator of microtubule dynamics...,24.819050,NaN,0.008621,23.186899,999.0,0.013351,24.17310,50.688920,0.000456,27.525101,999.0,0.016516,999.0,HeLa cells,SILAC pulse-chase,Human


In [ ]:
data0['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data0 = data0[~(pd.isna(data0['Half-life']) | pd.isna(data0['Gene']))]
print(pd.to_numeric(data0['Half-life'], errors='coerce').notna().all())
data0.head()

data0 = data0[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data0.to_csv(save_as, index=False)

True


### Dataset 2

In [10]:
idx = 1
data1 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx])
data1.head()

File                                      Cambridge2011_C2C12_tableS1.csv
Type                                                                  Deg
Species                                                             Mouse
Cell_type                                                 C2C12 myoblasts
Method                                                  SILAC pulse-chase
Experiment_type                                                  In vitro
Quantity_description                                            Half-life
Calculation_method      Half-lives were calculated using the loss of “...
Notes                                                                 NaN
Path                    /mnt/lareaulab/reliscu/projects/Chronocell/dat...
Name: 1, dtype: object


,Protein IDs,Gene Names,Protein Descriptions,Uniprot,Proteins,Peptides (seq),Unique Peptides (seq),Mol. Weight [kDa],Ratio H/L,Ratio H/L Count,Intensity,half-life in h,kdeg,RIA
0,IPI00652107;IPI00651875;IPI00322513,Igfbp7;Mac25,"2 days neonate sympathetic ganglion cDNA, RIKE...",Q61581;Q3UFA6;A0A5E1;Q3U131;Q811C8,3.0,3.0,3.0,29.06,8.74,13.0,83773000.0,10.5,0.06598,0.3974
1,IPI00130758,Itm2b,Integral membrane protein 2B,O89051;Q3TVJ0;Q3UCT9;Q9CVA8;Q9D1Q3;Q9JME4,1.0,2.0,2.0,30.26,8.25,6.0,56180000.0,10.9,0.06381,0.3919
2,IPI00230349,Map1lc3b;Map1alc3;Map1lc3,Microtubule-associated proteins 1A/1B light ch...,Q9CQV6;Q2TBF9,1.0,3.0,1.0,14.62,7.84,8.0,123470000.0,11.2,0.06192,0.3869
3,IPI00119142,Appbp2;Kiaa0228,Amyloid protein-binding protein 2,Q9DAX9;Q5SX12,1.0,4.0,4.0,66.86,7.83,8.0,27175000.0,11.2,0.06188,0.3868
4,IPI00850611;IPI00775910;IPI00622700;IPI00460442,Brd2;Fsrg1;Kiaa4005;Ring3,similar to mKIAA4005 protein;Isoform 1 of Brom...,Q7JJ13-1;Q7JJ13;A2CFH2;Q7JJ13-2;A2CFH3,4.0,3.0,2.0,110.66,7.69,5.0,32304000.0,11.3,0.06123,0.3850


In [11]:
# Separate genes with multiple entries
data1['Gene Names'] = data1['Gene Names'].str.split(';')
data1 = data1.explode('Gene Names')
data1['Gene Names'] = data1['Gene Names'].str.strip()
data1 = data1.reset_index(drop=True)

# Standardize column names
data1['Gene'] = data1['Gene Names']
data1['Half-life'] = data1['half-life in h']
data1['Cell_type'] = legend1.loc[idx, 'Cell_type']
data1['Method'] = legend1.loc[idx, 'Method']
data1['Species'] = legend1.loc[idx, 'Species']

data1.head()

,Protein IDs,Gene Names,Protein Descriptions,Uniprot,Proteins,Peptides (seq),Unique Peptides (seq),Mol. Weight [kDa],Ratio H/L,Ratio H/L Count,Intensity,half-life in h,kdeg,RIA,Gene,Half-life,Cell_type,Method,Species
0,IPI00652107;IPI00651875;IPI00322513,Igfbp7,"2 days neonate sympathetic ganglion cDNA, RIKE...",Q61581;Q3UFA6;A0A5E1;Q3U131;Q811C8,3.0,3.0,3.0,29.06,8.74,13.0,83773000.0,10.5,0.06598,0.3974,Igfbp7,10.5,C2C12 myoblasts,SILAC pulse-chase,Mouse
1,IPI00652107;IPI00651875;IPI00322513,Mac25,"2 days neonate sympathetic ganglion cDNA, RIKE...",Q61581;Q3UFA6;A0A5E1;Q3U131;Q811C8,3.0,3.0,3.0,29.06,8.74,13.0,83773000.0,10.5,0.06598,0.3974,Mac25,10.5,C2C12 myoblasts,SILAC pulse-chase,Mouse
2,IPI00130758,Itm2b,Integral membrane protein 2B,O89051;Q3TVJ0;Q3UCT9;Q9CVA8;Q9D1Q3;Q9JME4,1.0,2.0,2.0,30.26,8.25,6.0,56180000.0,10.9,0.06381,0.3919,Itm2b,10.9,C2C12 myoblasts,SILAC pulse-chase,Mouse
3,IPI00230349,Map1lc3b,Microtubule-associated proteins 1A/1B light ch...,Q9CQV6;Q2TBF9,1.0,3.0,1.0,14.62,7.84,8.0,123470000.0,11.2,0.06192,0.3869,Map1lc3b,11.2,C2C12 myoblasts,SILAC pulse-chase,Mouse
4,IPI00230349,Map1alc3,Microtubule-associated proteins 1A/1B light ch...,Q9CQV6;Q2TBF9,1.0,3.0,1.0,14.62,7.84,8.0,123470000.0,11.2,0.06192,0.3869,Map1alc3,11.2,C2C12 myoblasts,SILAC pulse-chase,Mouse


In [ ]:
data1['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File']) 

# Misc clean up
data1 = data1[~(pd.isna(data1['Half-life']) | pd.isna(data1['Gene']))]
print(pd.to_numeric(data1['Half-life'], errors='coerce').notna().all())
data1.head()

data1 = data1[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data1.to_csv(save_as, index=False)

True


### Dataset 3

In [13]:
idx = 2
data2 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx])
data2.head()

File                                       Cambridge2011_HeLa_tableS1.csv
Type                                                                  Deg
Species                                                             Human
Cell_type                                                      HeLa cells
Method                                                  SILAC pulse-chase
Experiment_type                                                  In vitro
Quantity_description                                            Half-life
Calculation_method      Half-lives were calculated using the loss of “...
Notes                                                                 NaN
Path                    /mnt/lareaulab/reliscu/projects/Chronocell/dat...
Name: 2, dtype: object


,Protein IDs,Gene Names,Protein Descriptions,Uniprot,Proteins,Peptides (seq),Unique Peptides (seq),Mol. Weight [kDa],Ratio H/L,Ratio H/L Count,Intensity,half-life in h,kdeg,RIA
0,IPI00298409,PDCL,Phosducin-like protein,Q13371;Q4VXB6;Q9UNX1,1,2,2,34.28,14.65,4,5860100,8.1,0.08572,0.4361
1,IPI00299214;IPI00791117;IPI00794651;IPI00795236,TK1,"thymidine kinase 1, soluble;29 kDa protein;20 ...",P04183,4,3,3,29.89,12.76,5,9627000,8.6,0.08034,0.4273
2,IPI00179328;IPI00549924;IPI00000163;IPI00643624,CTNNAL1;RP11-115J22.1-005,Isoform 1 of Alpha-catulin;Isoform 2 of Alpha-...,Q9UBT7-1;Q9UBT7;Q9UBT7-2;Q9UBT7-3;Q5JTQ9;Q5JTQ6,4,4,4,81.90,12.40,7,12889000,8.7,0.07926,0.4254
3,IPI00847321;IPI00022748,TEGT;BI1;TMBIM6,testis enhanced gene transcript (BAX inhibitor...,A8MUR1;P55061,2,2,2,32.71,12.12,3,4025200,8.8,0.07837,0.4238
4,IPI00013241,UBL5,Ubiquitin-like protein 5,Q9BZL1,1,2,2,8.55,11.88,3,892020,8.9,0.07761,0.4224


In [14]:
# Separate genes with multiple entries
data2['Gene Names'] = data2['Gene Names'].str.split(';')
data2 = data2.explode('Gene Names')
data2['Gene Names'] = data2['Gene Names'].str.strip()
data2 = data2.reset_index(drop=True)

# Standardize column names
data2['Gene'] = data2['Gene Names']
data2['Half-life'] = data2['half-life in h']
data2['Cell_type'] = legend1.loc[idx, 'Cell_type']
data2['Method'] = legend1.loc[idx, 'Method']
data2['Species'] = legend1.loc[idx, 'Species']

data2.head()

,Protein IDs,Gene Names,Protein Descriptions,Uniprot,Proteins,Peptides (seq),Unique Peptides (seq),Mol. Weight [kDa],Ratio H/L,Ratio H/L Count,Intensity,half-life in h,kdeg,RIA,Gene,Half-life,Cell_type,Method,Species
0,IPI00298409,PDCL,Phosducin-like protein,Q13371;Q4VXB6;Q9UNX1,1,2,2,34.28,14.65,4,5860100,8.1,0.08572,0.4361,PDCL,8.1,HeLa cells,SILAC pulse-chase,Human
1,IPI00299214;IPI00791117;IPI00794651;IPI00795236,TK1,"thymidine kinase 1, soluble;29 kDa protein;20 ...",P04183,4,3,3,29.89,12.76,5,9627000,8.6,0.08034,0.4273,TK1,8.6,HeLa cells,SILAC pulse-chase,Human
2,IPI00179328;IPI00549924;IPI00000163;IPI00643624,CTNNAL1,Isoform 1 of Alpha-catulin;Isoform 2 of Alpha-...,Q9UBT7-1;Q9UBT7;Q9UBT7-2;Q9UBT7-3;Q5JTQ9;Q5JTQ6,4,4,4,81.90,12.40,7,12889000,8.7,0.07926,0.4254,CTNNAL1,8.7,HeLa cells,SILAC pulse-chase,Human
3,IPI00179328;IPI00549924;IPI00000163;IPI00643624,RP11-115J22.1-005,Isoform 1 of Alpha-catulin;Isoform 2 of Alpha-...,Q9UBT7-1;Q9UBT7;Q9UBT7-2;Q9UBT7-3;Q5JTQ9;Q5JTQ6,4,4,4,81.90,12.40,7,12889000,8.7,0.07926,0.4254,RP11-115J22.1-005,8.7,HeLa cells,SILAC pulse-chase,Human
4,IPI00847321;IPI00022748,TEGT,testis enhanced gene transcript (BAX inhibitor...,A8MUR1;P55061,2,2,2,32.71,12.12,3,4025200,8.8,0.07837,0.4238,TEGT,8.8,HeLa cells,SILAC pulse-chase,Human


In [ ]:
data2['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data2 = data2[~(pd.isna(data2['Half-life']) | pd.isna(data2['Gene']))]
print(pd.to_numeric(data2['Half-life'], errors='coerce').notna().all())
data2.head()

data2 = data2[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data2.to_csv(save_as, index=False)

True


In [16]:
# idx = 3
# data3 = pd.read_csv(legend1.loc[idx, 'Path'])
# print(legend1.loc[idx])
# data3.head()

In [17]:
# Skipping mouse brain data for now

### Dataset 6

In [18]:
idx = 5
data5 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx])
data5.head()

File                                           Fornasiero2018_tableS1.csv
Type                                                                  Deg
Species                                                             Mouse
Cell_type                                                           Heart
Method                                                      Dynamic SILAC
Experiment_type                                                   In vivo
Quantity_description                                            Half-life
Calculation_method      Expontential fitting (derived from solutions t...
Notes                   Pulsed for 0, 5, 14, and 21 days, and 30 and 6...
Path                    /mnt/lareaulab/reliscu/projects/Chronocell/dat...
Name: 5, dtype: object


,Uniprot identifier,Uniprot entry name,Gene name(s),Protein name(s),Protein half-life in brain cortex homogenate control (days),Protein half-life in brain cortex homogenate env. enrich. (days),Protein half-life in cerebellum homogenate control (days),Protein half-life in brain cortex synaptosomes control (days),Protein half-life in brain cortex synaptosomes env. enrich. (days),Protein half-life in cerebellum synaptosomes control (days),...,Cerebellum control synaptosomes Sequence coverage [%],Cortex control synaptic vesicles Peptide counts (razor+unique),Cortex control synaptic vesicles Peptide counts (unique),Cortex control synaptic vesicles Sequence coverage [%],Cortex EE SVs Peptide counts (razor+unique),Cortex EE SVs Peptide counts (unique),Cortex EE SVs Sequence coverage [%],Cerebellum control SVs Peptide counts (razor+unique),Cerebellum control SVs Peptide counts (unique),Cerebellum control SVs Sequence coverage [%]
0,O08553,DPYL2_MOUSE,Dpysl2 Crmp2 Ulip2,Dihydropyrimidinase-related protein 2 (DRP-2) ...,26.13,24.87,23.72,32.97,33.22,24.61,...,82.9,50,44,83.4,50,44,83.4,50,44,83.4
1,O35643,AP1B1_MOUSE,Ap1b1 Adtb1,AP-1 complex subunit beta-1 (Adaptor protein c...,12.54,12.03,11.93,16.94,16.46,15.31,...,63.2,40,40,68.7,40,40,68.7,40,40,68.7
2,O54734,OST48_MOUSE,Ddost,Dolichyl-diphosphooligosaccharide--protein gly...,7.01,5.64,7.20,6.53,5.43,8.82,...,49.0,17,17,61.9,17,17,61.9,17,17,61.9
3,O55131,SEPT7_MOUSE,Sept7 Cdc10,Septin-7 (CDC10 protein homolog),17.49,16.07,16.42,18.55,15.85,16.08,...,72.0,40,37,67.7,40,37,67.7,40,37,67.7
4,O55143,AT2A2_MOUSE,Atp2a2,Sarcoplasmic/endoplasmic reticulum calcium ATP...,5.92,5.66,5.84,6.06,6.06,6.22,...,55.3,64,42,55.4,64,42,55.4,64,42,55.4


In [19]:
data5['Half-life'] = data5.filter(regex='heart')
# Change units to hours
data5['Half-life'] = data5['Half-life'] * 24

# Separate rows with multiple genes
data5['Gene name(s)'] = data5['Gene name(s)'].str.split(" ")
data5 = data5.explode('Gene name(s)')
data5['Gene name(s)'] = data5['Gene name(s)'].str.strip()
data5 = data5.reset_index(drop=True)

# Standardize column names
data5['Gene'] = data5['Gene name(s)']
data5['Cell_type'] = legend1.loc[idx, 'Cell_type']
data5['Method'] = legend1.loc[idx, 'Method']
data5['Species'] = legend1.loc[idx, 'Species']

data5.head()

,Uniprot identifier,Uniprot entry name,Gene name(s),Protein name(s),Protein half-life in brain cortex homogenate control (days),Protein half-life in brain cortex homogenate env. enrich. (days),Protein half-life in cerebellum homogenate control (days),Protein half-life in brain cortex synaptosomes control (days),Protein half-life in brain cortex synaptosomes env. enrich. (days),Protein half-life in cerebellum synaptosomes control (days),...,Cortex EE SVs Peptide counts (unique),Cortex EE SVs Sequence coverage [%],Cerebellum control SVs Peptide counts (razor+unique),Cerebellum control SVs Peptide counts (unique),Cerebellum control SVs Sequence coverage [%],Half-life,Gene,Cell_type,Method,Species
0,O08553,DPYL2_MOUSE,Dpysl2,Dihydropyrimidinase-related protein 2 (DRP-2) ...,26.13,24.87,23.72,32.97,33.22,24.61,...,44,83.4,50,44,83.4,126.00,Dpysl2,Heart,Dynamic SILAC,Mouse
1,O08553,DPYL2_MOUSE,Crmp2,Dihydropyrimidinase-related protein 2 (DRP-2) ...,26.13,24.87,23.72,32.97,33.22,24.61,...,44,83.4,50,44,83.4,126.00,Crmp2,Heart,Dynamic SILAC,Mouse
2,O08553,DPYL2_MOUSE,Ulip2,Dihydropyrimidinase-related protein 2 (DRP-2) ...,26.13,24.87,23.72,32.97,33.22,24.61,...,44,83.4,50,44,83.4,126.00,Ulip2,Heart,Dynamic SILAC,Mouse
3,O35643,AP1B1_MOUSE,Ap1b1,AP-1 complex subunit beta-1 (Adaptor protein c...,12.54,12.03,11.93,16.94,16.46,15.31,...,40,68.7,40,40,68.7,83.28,Ap1b1,Heart,Dynamic SILAC,Mouse
4,O35643,AP1B1_MOUSE,Adtb1,AP-1 complex subunit beta-1 (Adaptor protein c...,12.54,12.03,11.93,16.94,16.46,15.31,...,40,68.7,40,40,68.7,83.28,Adtb1,Heart,Dynamic SILAC,Mouse


In [ ]:
data5['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data5 = data5[~(pd.isna(data5['Half-life']) | pd.isna(data5['Gene']))]
print(pd.to_numeric(data5['Half-life'], errors='coerce').notna().all())
data5.head()

data5 = data5[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data5.to_csv(save_as, index=False)

True


### Dataset 7

In [21]:
# Leg muscle

idx = 6
data6 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx])

data6['Half-life'] = data6.filter(regex='muscle')
# Change units to hours
data6['Half-life'] = data6['Half-life'] * 24

# Separate rows with multiple genes
data6['Gene name(s)'] = data6['Gene name(s)'].str.split(" ")
data6 = data6.explode('Gene name(s)')
data6['Gene name(s)'] = data6['Gene name(s)'].str.strip()
data6 = data6.reset_index(drop=True)

# Standardize column names
data6['Gene'] = data6['Gene name(s)']
data6['Cell_type'] = legend1.loc[idx, 'Cell_type']
data6['Method'] = legend1.loc[idx, 'Method']
data6['Species'] = legend1.loc[idx, 'Species']

data6.head()


File                                           Fornasiero2018_tableS1.csv
Type                                                                  Deg
Species                                                             Mouse
Cell_type                                                      Leg-muscle
Method                                                      Dynamic SILAC
Experiment_type                                                   In vivo
Quantity_description                                            Half-life
Calculation_method      Expontential fitting (derived from solutions t...
Notes                   Pulsed for 0, 5, 14, and 21 days, and 30 and 6...
Path                    /mnt/lareaulab/reliscu/projects/Chronocell/dat...
Name: 6, dtype: object


,Uniprot identifier,Uniprot entry name,Gene name(s),Protein name(s),Protein half-life in brain cortex homogenate control (days),Protein half-life in brain cortex homogenate env. enrich. (days),Protein half-life in cerebellum homogenate control (days),Protein half-life in brain cortex synaptosomes control (days),Protein half-life in brain cortex synaptosomes env. enrich. (days),Protein half-life in cerebellum synaptosomes control (days),...,Cortex EE SVs Peptide counts (unique),Cortex EE SVs Sequence coverage [%],Cerebellum control SVs Peptide counts (razor+unique),Cerebellum control SVs Peptide counts (unique),Cerebellum control SVs Sequence coverage [%],Half-life,Gene,Cell_type,Method,Species
0,O08553,DPYL2_MOUSE,Dpysl2,Dihydropyrimidinase-related protein 2 (DRP-2) ...,26.13,24.87,23.72,32.97,33.22,24.61,...,44,83.4,50,44,83.4,7.92,Dpysl2,Leg-muscle,Dynamic SILAC,Mouse
1,O08553,DPYL2_MOUSE,Crmp2,Dihydropyrimidinase-related protein 2 (DRP-2) ...,26.13,24.87,23.72,32.97,33.22,24.61,...,44,83.4,50,44,83.4,7.92,Crmp2,Leg-muscle,Dynamic SILAC,Mouse
2,O08553,DPYL2_MOUSE,Ulip2,Dihydropyrimidinase-related protein 2 (DRP-2) ...,26.13,24.87,23.72,32.97,33.22,24.61,...,44,83.4,50,44,83.4,7.92,Ulip2,Leg-muscle,Dynamic SILAC,Mouse
3,O35643,AP1B1_MOUSE,Ap1b1,AP-1 complex subunit beta-1 (Adaptor protein c...,12.54,12.03,11.93,16.94,16.46,15.31,...,40,68.7,40,40,68.7,30.48,Ap1b1,Leg-muscle,Dynamic SILAC,Mouse
4,O35643,AP1B1_MOUSE,Adtb1,AP-1 complex subunit beta-1 (Adaptor protein c...,12.54,12.03,11.93,16.94,16.46,15.31,...,40,68.7,40,40,68.7,30.48,Adtb1,Leg-muscle,Dynamic SILAC,Mouse


In [ ]:
data6['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data6 = data6[~(pd.isna(data6['Half-life']) | pd.isna(data6['Gene']))]
print(pd.to_numeric(data6['Half-life'], errors='coerce').notna().all())
data6.head()

data6 = data6[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data6.to_csv(save_as, index=False)

True


### Dataset 8

In [23]:
idx = 7
data7 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Cell_type'])

# HCT116 colon cancer-derived epithelial cells

# Standardize column names
data7['Gene'] = data7['Gene symbol']
data7['Cell_type'] = legend1.loc[idx, 'Cell_type']
data7['Method'] = legend1.loc[idx, 'Method']
data7['Species'] = legend1.loc[idx, 'Species']
data7.head()

HCT116 colon cancer-derived epithelial cells


,Uniprot ID,Gene symbol,CHX_0,CHX_0.1,CHX_0.2,CHX_0.3,CHX_1,CHX_1.1,CHX_2,CHX_2.1,...,CHX_8.2,CHX_8.3,Model,Half-life,R2,Short-lived,Gene,Cell_type,Method,Species
0,Q9Y3B1,PRELID3B,0.96,0.99,1.00,1.05,0.14,0.20,0.09,0.06,...,0.09,0.10,non-linear model,0.33,1.00,short-lived,PRELID3B,HCT116 colon cancer-derived epithelial cells,CHX chase,Human
1,Q14494,NFE2L1,0.90,1.03,1.02,1.04,0.17,0.19,0.12,0.16,...,0.08,0.08,non-linear model,0.37,0.99,short-lived,NFE2L1,HCT116 colon cancer-derived epithelial cells,CHX chase,Human
2,Q13794,PMAIP1,0.93,1.00,1.02,1.05,0.31,0.34,0.24,0.24,...,0.34,0.30,non-linear model,0.43,0.98,short-lived,PMAIP1,HCT116 colon cancer-derived epithelial cells,CHX chase,Human
3,P01106-2,MYC,0.92,0.90,1.06,1.11,0.22,0.24,0.17,0.18,...,0.09,0.10,non-linear model,0.44,0.98,short-lived,MYC,HCT116 colon cancer-derived epithelial cells,CHX chase,Human
4,Q15828,CST6,1.07,0.94,1.04,0.95,0.25,0.25,0.07,0.11,...,0.11,0.10,non-linear model,0.44,0.99,short-lived,CST6,HCT116 colon cancer-derived epithelial cells,CHX chase,Human


In [ ]:
data7['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data7 = data7[~(pd.isna(data7['Half-life']) | pd.isna(data7['Gene']))]
print(pd.to_numeric(data7['Half-life'], errors='coerce').notna().all())
data7.head()

data7 = data7[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data7.to_csv(save_as, index=False)

True


### Dataset 9

In [25]:
# HEK293T kidney cells

idx = 8
data8 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Cell_type'])

# Standardize column names
data8['Gene'] = data8['Gene symbol']
data8['Cell_type'] = legend1.loc[idx, 'Cell_type']
data8['Method'] = legend1.loc[idx, 'Method']
data8['Species'] = legend1.loc[idx, 'Species']
data8.head()

HEK293T kidney cells


,Uniprot ID,Gene symbol,CHX_0,CHX_0.1,CHX_0.2,CHX_0.3,CHX_1,CHX_1.1,CHX_2,CHX_2.1,...,CHX_8.2,CHX_8.3,Model,Half-life,R2,Short-lived,Gene,Cell_type,Method,Species
0,P30304,CDC25A,0.96,0.98,0.91,1.15,0.25,0.22,0.26,0.25,...,0.22,0.23,non-linear model,0.20,0.98,short-lived,CDC25A,HEK293T kidney cells,CHX chase,Human
1,Q9Y3B1,PRELID3B,0.98,0.97,0.83,1.22,0.13,0.19,0.10,0.07,...,0.08,0.07,non-linear model,0.30,0.96,short-lived,PRELID3B,HEK293T kidney cells,CHX chase,Human
2,P51959,CCNG1,0.98,0.97,1.01,1.04,0.33,0.28,0.29,0.29,...,0.23,0.24,non-linear model,0.44,0.99,short-lived,CCNG1,HEK293T kidney cells,CHX chase,Human
3,Q8IYN2,TCEAL8,0.98,1.04,1.03,0.95,0.23,0.23,0.10,0.08,...,0.04,0.05,non-linear model,0.46,1.00,short-lived,TCEAL8,HEK293T kidney cells,CHX chase,Human
4,Q6PEV8,FAM199X,1.00,0.96,0.95,1.09,0.28,0.23,0.09,0.10,...,0.04,0.06,non-linear model,0.47,0.99,short-lived,FAM199X,HEK293T kidney cells,CHX chase,Human


In [ ]:
data8['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data8 = data8[~(pd.isna(data8['Half-life']) | pd.isna(data8['Gene']))]
print(pd.to_numeric(data8['Half-life'], errors='coerce').notna().all())
data8.head()

data8 = data8[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data8.to_csv(save_as, index=False)

True


### Dataset 10 

In [27]:
# RPE-1 retinal pigmented epithelial cells

idx = 9
data9 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Cell_type'])

# Standardize column names
data9['Gene'] = data9['Gene symbol']
data9['Cell_type'] = legend1.loc[idx, 'Cell_type']
data9['Method'] = legend1.loc[idx, 'Method']
data9['Species'] = legend1.loc[idx, 'Species']
data9.head()

RPE-1 retinal pigmented epithelial cells


,Uniprot ID,Gene symbol,CHX_0,CHX_0.1,CHX_0.2,CHX_0.3,CHX_1,CHX_1.1,CHX_2,CHX_2.1,...,CHX_8.2,CHX_8.3,Model,Half-life,R2,Short-lived,Gene,Cell_type,Method,Species
0,P57086,SCAND1,1.18,0.86,0.99,0.97,0.17,0.28,0.20,0.12,...,0.21,0.19,non-linear model,0.34,0.97,short-lived,SCAND1,RPE-1 retinal pigmented epithelial cells,CHX chase,Human
1,P24385,CCND1,1.39,0.73,0.53,1.35,0.28,0.16,0.11,0.10,...,0.08,0.07,non-linear model,0.42,0.81,short-lived,CCND1,RPE-1 retinal pigmented epithelial cells,CHX chase,Human
2,P52823,STC1,1.02,0.86,1.15,0.97,0.31,0.37,0.26,0.23,...,0.46,0.26,non-linear model,0.44,0.94,short-lived,STC1,RPE-1 retinal pigmented epithelial cells,CHX chase,Human
3,O76061,STC2,1.04,0.91,1.05,1.00,0.37,0.31,0.25,0.26,...,0.29,0.31,non-linear model,0.47,0.99,short-lived,STC2,RPE-1 retinal pigmented epithelial cells,CHX chase,Human
4,Q8IYN2,TCEAL8,0.83,0.92,1.09,1.16,0.07,0.01,0.07,0.12,...,0.08,0.00,manual inspection,0.52,1.00,short-lived,TCEAL8,RPE-1 retinal pigmented epithelial cells,CHX chase,Human


In [ ]:
data9['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data9 = data9[~(pd.isna(data9['Half-life']) | pd.isna(data9['Gene']))]
print(pd.to_numeric(data9['Half-life'], errors='coerce').notna().all())
data9.head()

data9 = data9[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data9.to_csv(save_as, index=False)

True


### Dataset 11

In [29]:
# U2OS tibia cells

idx = 10
data10 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Cell_type'])

# Standardize column names
data10['Gene'] = data10['Gene symbol']
data10['Cell_type'] = legend1.loc[idx, 'Cell_type']
data10['Method'] = legend1.loc[idx, 'Method']
data10['Species'] = legend1.loc[idx, 'Species']
data10.head()

U2OS tibia cells


,Uniprot ID,Gene symbol,CHX_0,CHX_0.1,CHX_0.2,CHX_0.3,CHX_1,CHX_1.1,CHX_2,CHX_2.1,...,CHX_8.2,CHX_8.3,Model,Half-life,R2,Short-lived,Gene,Cell_type,Method,Species
0,Q9Y3B1,PRELID3B,1.08,0.96,0.98,0.98,0.30,0.19,0.13,0.24,...,0.21,0.15,non-linear model,0.34,0.99,short-lived,PRELID3B,U2OS tibia cells,CHX chase,Human
1,Q96E09,FAM122A,1.09,0.93,0.96,1.02,0.28,0.24,0.11,0.16,...,0.18,0.10,non-linear model,0.43,0.99,short-lived,FAM122A,U2OS tibia cells,CHX chase,Human
2,P30279,CCND2,1.06,0.98,1.02,0.93,0.27,0.17,0.11,0.08,...,0.08,0.08,non-linear model,0.43,0.99,short-lived,CCND2,U2OS tibia cells,CHX chase,Human
3,Q9Y6B2,EID1,1.02,0.99,0.98,1.01,0.22,0.21,0.09,0.09,...,0.05,0.06,non-linear model,0.43,1.00,short-lived,EID1,U2OS tibia cells,CHX chase,Human
4,Q8NA92,THAP8,0.91,0.92,1.13,1.04,0.30,0.36,0.29,0.31,...,0.34,0.34,non-linear model,0.46,0.96,short-lived,THAP8,U2OS tibia cells,CHX chase,Human


In [ ]:
data10['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data10 = data10[~(pd.isna(data10['Half-life']) | pd.isna(data10['Gene']))]
print(pd.to_numeric(data10['Half-life'], errors='coerce').notna().all())
data10.head()

data10 = data10[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data10.to_csv(save_as, index=False)

True


### Dataset 12

In [31]:
idx = 11
data11 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Cell_type'])
data11.head()

B cells


,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Hepatocytes replicate 1 R_sq,Hepatocytes replicate 2 half_life,Hepatocytes replicate 2 dataQual,Hepatocytes replicate 2 R_sq,Monocytes replicate 1 half_life,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.885,24.377,good,0.905,14.599,weak,0.978,NaN,NaN,NaN
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,NaN,145.343,weak,0.852,78.090,weak,0.995,57.957,weak,0.987
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,69.614,weak,0.950,52.862,weak,0.949
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,76.276,weak,0.982,59.539,good,0.988
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,68.136,weak,0.938,51.380,weak,0.985


In [32]:
# Average half-life values over replicates for B cells
pattern = r'^Bcells replicate \d+ half_life'  # regex for exact match with digits at end
matching_cols = data11.filter(regex=pattern).columns
data11['Half-life'] = data11[matching_cols].mean(axis=1)

# Standardize column names
data11['Gene'] = data11['gene_name']
data11['Cell_type'] = legend1.loc[idx, 'Cell_type']
data11['Method'] = legend1.loc[idx, 'Method']
data11['Species'] = legend1.loc[idx, 'Species']

data11.head()

,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq,Half-life,Gene,Cell_type,Method,Species
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.978,NaN,NaN,NaN,NaN,A2M,B cells,Dynamic SILAC,Human
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,weak,0.995,57.957,weak,0.987,81.8375,AAAS,B cells,Dynamic SILAC,Human
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,weak,0.950,52.862,weak,0.949,30.5240,AACS,B cells,Dynamic SILAC,Human
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.982,59.539,good,0.988,NaN,AAK1,B cells,Dynamic SILAC,Human
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.938,51.380,weak,0.985,NaN,AAMDC,B cells,Dynamic SILAC,Human


In [33]:
data11[matching_cols]

,Bcells replicate 1 half_life,Bcells replicate 2 half_life
0,NaN,NaN
1,75.299,88.376
2,NaN,30.524
3,NaN,NaN
4,NaN,NaN
...,...,...
8799,NaN,NaN
8800,NaN,NaN
8801,NaN,NaN
8802,NaN,NaN


In [ ]:
data11['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data11 = data11[~(pd.isna(data11['Half-life']) | pd.isna(data11['Gene']))]
print(pd.to_numeric(data11['Half-life'], errors='coerce').notna().all())
data11.head()

data11 = data11[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data11.to_csv(save_as, index=False)

True


### Dataset 13

In [35]:
# Monocytes

idx = 12
data12 = pd.read_csv(legend1.loc[idx, 'Path'])

# Average half-life values over replicates
pattern = r'^Monocytes replicate \d+ half_life'  # regex for exact match with digits at end
matching_cols = data12.filter(regex=pattern).columns
data12['Half-life'] = data12[matching_cols].mean(axis=1)

print(legend1.loc[idx, 'Cell_type'])

# Standardize column names
data12['Gene'] = data12['gene_name']
data12['Cell_type'] = legend1.loc[idx, 'Cell_type']
data12['Method'] = legend1.loc[idx, 'Method']
data12['Species'] = legend1.loc[idx, 'Species']

data12.head()

Monocytes


,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq,Half-life,Gene,Cell_type,Method,Species
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.978,NaN,NaN,NaN,14.5990,A2M,Monocytes,Dynamic SILAC,Human
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,weak,0.995,57.957,weak,0.987,68.0235,AAAS,Monocytes,Dynamic SILAC,Human
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,weak,0.950,52.862,weak,0.949,61.2380,AACS,Monocytes,Dynamic SILAC,Human
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.982,59.539,good,0.988,67.9075,AAK1,Monocytes,Dynamic SILAC,Human
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.938,51.380,weak,0.985,59.7580,AAMDC,Monocytes,Dynamic SILAC,Human


In [36]:
data12[matching_cols]

,Monocytes replicate 1 half_life,Monocytes replicate 2 half_life
0,14.599,NaN
1,78.090,57.957
2,69.614,52.862
3,76.276,59.539
4,68.136,51.380
...,...,...
8799,NaN,NaN
8800,NaN,NaN
8801,NaN,NaN
8802,NaN,NaN


In [ ]:
data12['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data12 = data12[~(pd.isna(data12['Half-life']) | pd.isna(data12['Gene']))]
print(pd.to_numeric(data12['Half-life'], errors='coerce').notna().all())
data12.head()

data12 = data12[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data12.to_csv(save_as, index=False)

True


### Dataset 14

In [38]:
# NK cells

idx = 13
data13 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Cell_type'])

# Average half-life values over replicates
pattern = r'^NK cells replicate \d+ half_life'  # regex for exact match with digits at end
matching_cols = data13.filter(regex=pattern).columns
data13['Half-life'] = data13[matching_cols].mean(axis=1)

# Standardize column names
data13['Gene'] = data13['gene_name']
data13['Cell_type'] = legend1.loc[idx, 'Cell_type']
data13['Method'] = legend1.loc[idx, 'Method']
data13['Species'] = legend1.loc[idx, 'Species']

data13.head()

NK cells


,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq,Half-life,Gene,Cell_type,Method,Species
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.978,NaN,NaN,NaN,NaN,A2M,NK cells,Dynamic SILAC,Human
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,weak,0.995,57.957,weak,0.987,293.8175,AAAS,NK cells,Dynamic SILAC,Human
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,weak,0.950,52.862,weak,0.949,98.5740,AACS,NK cells,Dynamic SILAC,Human
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.982,59.539,good,0.988,152.6780,AAK1,NK cells,Dynamic SILAC,Human
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.938,51.380,weak,0.985,NaN,AAMDC,NK cells,Dynamic SILAC,Human


In [39]:
data13[matching_cols]

,NK cells replicate 1 half_life,NK cells replicate 2 half_life
0,NaN,NaN
1,379.453,208.182
2,NaN,98.574
3,NaN,152.678
4,NaN,NaN
...,...,...
8799,NaN,NaN
8800,NaN,NaN
8801,NaN,NaN
8802,NaN,NaN


In [ ]:
data13['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data13 = data13[~(pd.isna(data13['Half-life']) | pd.isna(data13['Gene']))]
print(pd.to_numeric(data13['Half-life'], errors='coerce').notna().all())
data13.head()

data13 = data13[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data13.to_csv(save_as, index=False)

True


### Dataset 15

In [41]:
# Hepatocytes

idx = 14
data14 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Cell_type'])

# Average half-life values over replicates
pattern = r'^Hepatocytes replicate \d+ half_life'  # regex for exact match with digits at end
matching_cols = data14.filter(regex=pattern).columns
data14['Half-life'] = data14[matching_cols].mean(axis=1)

# Standardize column names
data14['Gene'] = data14['gene_name']
data14['Cell_type'] = legend1.loc[idx, 'Cell_type']
data14['Method'] = legend1.loc[idx, 'Method']
data14['Species'] = legend1.loc[idx, 'Species']

data14.head()

Hepatocytes


,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq,Half-life,Gene,Cell_type,Method,Species
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.978,NaN,NaN,NaN,24.0255,A2M,Hepatocytes,Dynamic SILAC,Human
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,weak,0.995,57.957,weak,0.987,145.3430,AAAS,Hepatocytes,Dynamic SILAC,Human
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,weak,0.950,52.862,weak,0.949,NaN,AACS,Hepatocytes,Dynamic SILAC,Human
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.982,59.539,good,0.988,NaN,AAK1,Hepatocytes,Dynamic SILAC,Human
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,weak,0.938,51.380,weak,0.985,NaN,AAMDC,Hepatocytes,Dynamic SILAC,Human


In [42]:
data14[matching_cols]

,Hepatocytes replicate 1 half_life,Hepatocytes replicate 2 half_life
0,23.674,24.377
1,NaN,145.343
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
8799,NaN,NaN
8800,NaN,NaN
8801,NaN,NaN
8802,NaN,NaN


In [ ]:
data14['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data14 = data14[~(pd.isna(data14['Half-life']) | pd.isna(data14['Gene']))]
print(pd.to_numeric(data14['Half-life'], errors='coerce').notna().all())
data14.head()

data14 = data14[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data14.to_csv(save_as, index=False)

True


### Dataset 16

In [44]:
# Embryonic neurons 

idx = 15
data15 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Cell_type'])

# Average half-life values over replicates
pattern = r'^Mouse Neurons, replicate \d+ half_life'  # regex for exact match with digits at end
matching_cols = data15.filter(regex=pattern).columns
data15['Half-life'] = data15[matching_cols].mean(axis=1)

# Standardize column names
data15['Gene'] = data15['gene_name']
data15['Cell_type'] = legend1.loc[idx, 'Cell_type']
data15['Method'] = legend1.loc[idx, 'Method']
data15['Species'] = legend1.loc[idx, 'Species']

data15.head()

Embryonic neurons


,gene_name,"Mouse Neurons, replicate 3 half_life","Mouse Neurons, replicate 3 dataQual","Mouse Neurons, replicate 3 R_sq","Mouse Neurons, replicate 4 half_life","Mouse Neurons, replicate 4 dataQual","Mouse Neurons, replicate 4 R_sq",Half-life,Gene,Cell_type,Method,Species
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2M,Embryonic neurons,Dynamic SILAC,Mouse
1,AAAS,78.989,weak,0.982,86.873,good,0.975,82.9310,AAAS,Embryonic neurons,Dynamic SILAC,Mouse
2,AACS,28.209,good,1.000,32.865,good,1.000,30.5370,AACS,Embryonic neurons,Dynamic SILAC,Mouse
3,AAK1,34.468,good,0.999,43.131,good,0.997,38.7995,AAK1,Embryonic neurons,Dynamic SILAC,Mouse
4,AAMDC,NaN,NaN,NaN,15.508,weak,0.930,15.5080,AAMDC,Embryonic neurons,Dynamic SILAC,Mouse


In [45]:
data15[matching_cols]

,"Mouse Neurons, replicate 3 half_life","Mouse Neurons, replicate 4 half_life"
0,NaN,NaN
1,78.989,86.873
2,28.209,32.865
3,34.468,43.131
4,NaN,15.508
...,...,...
8799,NaN,17.394
8800,NaN,27.936
8801,NaN,59.059
8802,NaN,15.391


In [ ]:
data15['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data15 = data15[~(pd.isna(data15['Half-life']) | pd.isna(data15['Gene']))]
print(pd.to_numeric(data15['Half-life'], errors='coerce').notna().all())
data15.head()

data15 = data15[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data15.to_csv(save_as, index=False)

True


### Dataset 17

In [47]:
idx = 16
data16 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Species'])
data16.head()

Human


,gene_name,Sample,median_kdeg,median_Thalf,found_in_replicates
0,1110004f10rik|smap,mouse,0.020362,34.041064,3
1,1110004f10rik|smap,mouse_2DG,0.012545,55.251729,1
2,2210016l21rik|custos,mouse,0.040182,17.250261,3
3,2210016l21rik|custos,mouse_2DG,0.032057,21.622114,1
4,2310022a10rik,mouse,0.046491,14.909245,3


In [48]:
# Subset to human data
data16 = data16[data16['Sample'].str.contains('human')]

# Separate genes with multiple entries
data16['gene_name'] = data16['gene_name'].str.split('|')
data16 = data16.explode('gene_name')
data16['gene_name'] = data16['gene_name'].str.strip()
data16 = data16.reset_index(drop=True)

# Standardize columns
data16['Gene'] = data16['gene_name']
data16['Half-life'] = data16['median_Thalf']
data16['Cell_type'] = legend1.loc[idx, 'Cell_type']
data16['Method'] = legend1.loc[idx, 'Method']
data16['Species'] = legend1.loc[idx, 'Species']

data16.head()

,gene_name,Sample,median_kdeg,median_Thalf,found_in_replicates,Gene,Half-life,Cell_type,Method,Species
0,A0A0B4J2A0,human,0.009091,76.243850,3,A0A0B4J2A0,76.243850,Induced presomitic mesoderm cells,Dynamic SILAC,Human
1,SHPK,human,0.009091,76.243850,3,SHPK,76.243850,Induced presomitic mesoderm cells,Dynamic SILAC,Human
2,A0A2R8YDQ0,human,0.019075,36.338478,2,A0A2R8YDQ0,36.338478,Induced presomitic mesoderm cells,Dynamic SILAC,Human
3,VPS33B,human,0.019075,36.338478,2,VPS33B,36.338478,Induced presomitic mesoderm cells,Dynamic SILAC,Human
4,A0A2R8YFF1,human,0.051287,13.515126,2,A0A2R8YFF1,13.515126,Induced presomitic mesoderm cells,Dynamic SILAC,Human


In [ ]:
data16['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data16 = data16[~(pd.isna(data16['Half-life']) | pd.isna(data16['Gene']))]
print(pd.to_numeric(data16['Half-life'], errors='coerce').notna().all())
data16.head()

data16 = data16[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data16.to_csv(save_as, index=False)

True


### Dataset 18

In [50]:
# Subset to mouse data

idx = 17
data17 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Species'])
data17 = data17[data17['Sample'].str.contains('mouse$', regex=True)]

# Separate genes with multiple entries
data17['gene_name'] = data17['gene_name'].str.split('|')
data17 = data17.explode('gene_name')
data17['gene_name'] = data17['gene_name'].str.strip()
data17 = data17.reset_index(drop=True)

# Standardize columns
data17['Gene'] = data17['gene_name']
data17['Half-life'] = data17['median_Thalf']
data17['Cell_type'] = legend1.loc[idx, 'Cell_type']
data17['Method'] = legend1.loc[idx, 'Method']
data17['Species'] = legend1.loc[idx, 'Species']

data17.head()

Mouse


,gene_name,Sample,median_kdeg,median_Thalf,found_in_replicates,Gene,Half-life,Cell_type,Method,Species
0,1110004f10rik,mouse,0.020362,34.041064,3,1110004f10rik,34.041064,Induced presomitic mesoderm cells,Dynamic SILAC,Mouse
1,smap,mouse,0.020362,34.041064,3,smap,34.041064,Induced presomitic mesoderm cells,Dynamic SILAC,Mouse
2,2210016l21rik,mouse,0.040182,17.250261,3,2210016l21rik,17.250261,Induced presomitic mesoderm cells,Dynamic SILAC,Mouse
3,custos,mouse,0.040182,17.250261,3,custos,17.250261,Induced presomitic mesoderm cells,Dynamic SILAC,Mouse
4,2310022a10rik,mouse,0.046491,14.909245,3,2310022a10rik,14.909245,Induced presomitic mesoderm cells,Dynamic SILAC,Mouse


In [ ]:
data17['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data17 = data17[~(pd.isna(data17['Half-life']) | pd.isna(data17['Gene']))]
print(pd.to_numeric(data17['Half-life'], errors='coerce').notna().all())
data17.head()

data17 = data17[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data17.to_csv(save_as, index=False)

True


### Dataset 19

In [52]:
idx = 18
data18 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Species'])
data18.head()

Human


,Protein IDs (Uniprot),Gene names,Fasta header,Peptides,Unique peptides,Sequence coverage,Q-value,Rep1 tp = 0h,Rep1 tp = 1h,Rep1 tp = 2h,...,2-state-model_ktransfer [h-1],1-state-model_AICp,1-state-model_RSS,2-state-model_AICp,2-state-model_RSS,Half-life (exponential) 1-state-model [h],Half-life (steady state) 2-state-model [h],Δ-score,Degradation profile,Abundance after pulse
0,A0A024QZ33,NSRP1,>tr|A0A024QZ33|A0A024QZ33_HUMAN Coiled-coil do...,5,5,9.3,0.0,NaN,NaN,NaN,...,0.054953,0.999991,0.001396,0.000009,0.000360,7.47739764,> 300,0.086042,ED,0.024137
1,A0A075B730,EPPK1,>tr|A0A075B730|A0A075B730_HUMAN Epiplakin OS=H...,11,5,12.0,0.0,NaN,NaN,NaN,...,0.139357,0.999996,0.003928,0.000004,0.001967,11.04632441,> 300,0.062405,ED,0.003067
2,A0A087WSW9,TXNRD1,>tr|A0A087WSW9|A0A087WSW9_HUMAN Glycosyl-phosp...,24,23,65.3,0.0,100.0,94.079493,125.847867,...,0.693370,0.930673,0.032590,0.069327,0.032568,> 300,> 300,-0.002852,ED,1.334142
3,A0A087WTA5,EIF2B4,>tr|A0A087WTA5|A0A087WTA5_HUMAN Translation in...,13,13,30.6,0.0,NaN,NaN,NaN,...,3.183012,0.982396,0.031536,0.017604,0.022845,> 300,> 300,-0.087310,ED,0.025497
4,A0A087WTN3,SELT,>tr|A0A087WTN3|A0A087WTN3_HUMAN Selenoprotein ...,4,4,31.5,0.0,NaN,NaN,NaN,...,0.151134,0.999158,0.030894,0.000842,0.023324,14.49154835,7.6,-0.032537,ED,0.086726


In [53]:
# Use 2-state half-life estimates for proteins better modeled by this model
data18['Half-life'] = data18['Half-life (exponential) 1-state-model [h]']
use_2_state = data18['1-state-model_AICp'] < data18['2-state-model_AICp']
data18.loc[use_2_state, 'Half-life'] = data18.loc[use_2_state, 'Half-life (steady state) 2-state-model [h]']

data18.loc[data18['Half-life'] == "> 300", 'Half-life'] = 300

# Separate genes with multiple entries
data18['Gene names'] = data18['Gene names'].str.split(";")
data18 = data18.explode('Gene names')
data18['Gene names'] = data18['Gene names'].str.strip()
data18 = data18.reset_index(drop=True)

# Standardize columns
data18['Gene'] = data18['Gene names']	
data18['Cell_type'] = legend1.loc[idx, 'Cell_type']
data18['Method'] = legend1.loc[idx, 'Method']
data18['Species'] = legend1.loc[idx, 'Species']

data18.head()

,Protein IDs (Uniprot),Gene names,Fasta header,Peptides,Unique peptides,Sequence coverage,Q-value,Rep1 tp = 0h,Rep1 tp = 1h,Rep1 tp = 2h,...,Half-life (exponential) 1-state-model [h],Half-life (steady state) 2-state-model [h],Δ-score,Degradation profile,Abundance after pulse,Half-life,Gene,Cell_type,Method,Species
0,A0A024QZ33,NSRP1,>tr|A0A024QZ33|A0A024QZ33_HUMAN Coiled-coil do...,5,5,9.3,0.0,NaN,NaN,NaN,...,7.47739764,> 300,0.086042,ED,0.024137,7.47739764,NSRP1,RPE-1 retinal pigmented epithelial cells,SILAC + AHA pulse-chase,Human
1,A0A075B730,EPPK1,>tr|A0A075B730|A0A075B730_HUMAN Epiplakin OS=H...,11,5,12.0,0.0,NaN,NaN,NaN,...,11.04632441,> 300,0.062405,ED,0.003067,11.04632441,EPPK1,RPE-1 retinal pigmented epithelial cells,SILAC + AHA pulse-chase,Human
2,A0A087WSW9,TXNRD1,>tr|A0A087WSW9|A0A087WSW9_HUMAN Glycosyl-phosp...,24,23,65.3,0.0,100.0,94.079493,125.847867,...,> 300,> 300,-0.002852,ED,1.334142,300,TXNRD1,RPE-1 retinal pigmented epithelial cells,SILAC + AHA pulse-chase,Human
3,A0A087WTA5,EIF2B4,>tr|A0A087WTA5|A0A087WTA5_HUMAN Translation in...,13,13,30.6,0.0,NaN,NaN,NaN,...,> 300,> 300,-0.087310,ED,0.025497,300,EIF2B4,RPE-1 retinal pigmented epithelial cells,SILAC + AHA pulse-chase,Human
4,A0A087WTN3,SELT,>tr|A0A087WTN3|A0A087WTN3_HUMAN Selenoprotein ...,4,4,31.5,0.0,NaN,NaN,NaN,...,14.49154835,7.6,-0.032537,ED,0.086726,14.49154835,SELT,RPE-1 retinal pigmented epithelial cells,SILAC + AHA pulse-chase,Human


In [ ]:
data18['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data18 = data18[~(pd.isna(data18['Half-life']) | pd.isna(data18['Gene']))]
print(pd.to_numeric(data18['Half-life'], errors='coerce').notna().all())
data18.head()

data18 = data18[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data18.to_csv(save_as, index=False)

True


### Dataset 20

In [55]:
idx = 19
data19 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Species'])

# Use 2-state half-life estimates for proteins better modeled by this model
data19['Half-life'] = data19['Half-life (exponential) 1-state-model [h]']
use_2_state = data19['1-state-model_AICp'] < data19['2-state-model_AICp']
data19.loc[use_2_state, 'Half-life'] = data19.loc[use_2_state, 'Half-life (steady state) 2-state-model [h]']

data19.loc[data19['Half-life'] == "> 300", 'Half-life'] = 300

# Separate genes with multiple entries
data19['Gene names'] = data19['Gene names'].str.split(";")
data19 = data19.explode('Gene names')
data19['Gene names'] = data19['Gene names'].str.strip()
data19 = data19.reset_index(drop=True)

# Standardize columns
data19['Gene'] = data19['Gene names']	
data19['Cell_type'] = legend1.loc[idx, 'Cell_type']
data19['Method'] = legend1.loc[idx, 'Method']
data19['Species'] = legend1.loc[idx, 'Species']

data19.head()

Mouse


,Protein IDs (Uniprot),Gene names,Fasta header,Peptides,Unique peptides,Sequence coverage,Q-value,Rep1 tp = 0h,Rep1 tp = 1h,Rep1 tp = 2h,...,Half-life (steady state) 2-state-model [h],Δ-score,Degradation profile,Abundance after pulse,Validation score,Half-life,Gene,Cell_type,Method,Species
0,A2A432,Cul4b,>sp|A2A432|CUL4B_MOUSE Cullin-4B OS=Mus muscul...,45,32,48.9,0.0,100.0,117.756113,115.240505,...,32.6,0.040447,ED,0.158173,NaN,31.37826521,Cul4b,NIH3T3 fibroblasts,SILAC + AHA pulse-chase,Mouse
1,A2A4P0,Dhx8,>sp|A2A4P0|DHX8_MOUSE ATP-dependent RNA helica...,24,23,25.7,0.0,NaN,NaN,NaN,...,28.7,0.048949,UN,0.013817,NaN,28.7,Dhx8,NIH3T3 fibroblasts,SILAC + AHA pulse-chase,Mouse
2,A2A5K6,Znf335,>sp|A2A5K6|ZN335_MOUSE Zinc finger protein 335...,9,9,8.8,0.0,NaN,NaN,NaN,...,> 300,0.280086,UN,0.014009,NaN,3.599724224,Znf335,NIH3T3 fibroblasts,SILAC + AHA pulse-chase,Mouse
3,A2A5R2,Arfgef2,>sp|A2A5R2|BIG2_MOUSE Brefeldin A-inhibited gu...,21,11,15.9,0.0,NaN,NaN,NaN,...,> 300,-0.062858,ED,0.006254,NaN,24.37130152,Arfgef2,NIH3T3 fibroblasts,SILAC + AHA pulse-chase,Mouse
4,A2A6Q5-2,Cdc27,>sp|A2A6Q5-2|CDC27_MOUSE Isoform 2 of Cell div...,21,21,45.8,0.0,100.0,NaN,NaN,...,81.1,0.086752,UN,0.047189,NaN,21.05771266,Cdc27,NIH3T3 fibroblasts,SILAC + AHA pulse-chase,Mouse


In [ ]:
data19['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data19 = data19[~(pd.isna(data19['Half-life']) | pd.isna(data19['Gene']))]
print(pd.to_numeric(data19['Half-life'], errors='coerce').notna().all())
data19.head()

data19 = data19[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data19.to_csv(save_as, index=False)

True


### Dataset 21

In [57]:
idx = 20
data20 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Species'])
data20.head()

Mouse


,Protein IDs,Protein Names,Gene Names,Protein Descriptions,Uniprot IDs,RefSeq protein IDs,Refseq mRNA ID,ENSEMBL ID,MGI ID,Protein length [amino acids],...,Protein half-life average [h],mRNA half-life experiment [h],mRNA half-life replicate [h],mRNA half-life average [h],transcription rate (vsr) experiment [molecules/(cell*h)],transcription rate (vsr) replicate [molecules/(cell*h)],transcription rate (vsr) average [molecules/(cell*h)],translation rate constant (ksp) experiment [molecules/(mRNA*h)],translation rate constant (ksp) replicate [molecules/(mRNA*h)],translation rate constant (ksp) average [molecules/(mRNA*h)]
0,IPI00229543.3;IPI00221463.3;IPI00265761.6;IPI0...,Histone H2A type 1-F;Histone H2A type 3;Histon...,Hist1h2af;Hist3h2a;Hist1h2ab;Hist1h2ac;Hist1h2...,Histone H2A type 1-F;Histone H2A type 3;Histon...,Q8CGP5;Q61668;Q8BFU2;A2AB79;P22752;B2RVF0;Q8CG...,NP_783592;NP_835736;NP_783591;NP_835489;NP_835...,NaN,ENSMUST00000073261;ENSMUST00000108817;ENSMUST0...,MGI:2448309;MGI:2448458;MGI:2448306;MGI:244828...,130,...,94.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IPI00624840.2;IPI00623776.3;IPI00624933.1;IPI0...,Histone H4,Hist1h4a;Hist1h4b;H4-53;Hist1h4c;H4-12;Hist1h4...,12 kDa protein;similar to histone H4;12 kDa pr...,P62806;B2RTM0;Q61667;Q6B822,XP_981474;NP_291074;NP_694813;NP_783583;NP_783...,NaN,ENSMUST00000073009;ENSMUST00000075596;ENSMUST0...,MGI:2448419;MGI:2448420;MGI:2448421;MGI:244842...,105,...,104.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IPI00110850.1;IPI00473320.2;IPI00874780.1;IPI0...,"Actin, cytoplasmic 1;Beta-actin;Actin, cytopla...",Actb,"Actin, cytoplasmic 1;Putative uncharacterized ...",P60710;B2RRX1;Q3TIJ9;Q3U5R4;Q3UA89;Q3UAA9;Q3UA...,NP_031419,NM_007393,ENSMUST00000031564;ENSMUST00000100497,MGI:87904;MGI:87904,375,...,61.92,14.28,8.8,11.21,170.63,194.77,182.3,1506.65,2384.84,1895.55
3,IPI00348270.3;IPI00124518.3,Histone H2B type 2-B;H2b 616,Hist2h2bb,Histone H2B type 2-B,Q64525,NP_783597,NaN,ENSMUST00000018246;ENSMUST00000051091;ENSMUST0...,MGI:2448413,126,...,93.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IPI00282848.1;IPI00230730.4;IPI00659000.1;IPI0...,Histone H3.2,Hist1h3b;H3-53;H3.2;H3b;Hist1h3c;H3-143;Hist1h...,"histone cluster 2, H3c1;Histone H3.2;predicted...",P84228;A1L0U3;B9EI85,NP_835734;NP_038576;NP_473386;NP_783584;NP_835...,NaN,ENSMUST00000075558;ENSMUST00000090779;ENSMUST0...,MGI:2448319;MGI:2448320;MGI:2448322;MGI:244832...,181,...,79.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Separate rows with multiple genes
data20['Gene Names'] = data20['Gene Names'].str.split(";")
data20 = data20.explode('Gene Names')
data20['Gene Names'] = data20['Gene Names'].str.strip()
data20 = data20.reset_index(drop=True)

# Standardize columns
data20['Half-life'] = data20['Protein half-life average [h]']
data20['Gene'] = data20['Gene Names']	
data20['Cell_type'] = legend1.loc[idx, 'Cell_type']
data20['Method'] = legend1.loc[idx, 'Method']
data20['Species'] = legend1.loc[idx, 'Species']

data20.head()

,Protein IDs,Protein Names,Gene Names,Protein Descriptions,Uniprot IDs,RefSeq protein IDs,Refseq mRNA ID,ENSEMBL ID,MGI ID,Protein length [amino acids],...,transcription rate (vsr) replicate [molecules/(cell*h)],transcription rate (vsr) average [molecules/(cell*h)],translation rate constant (ksp) experiment [molecules/(mRNA*h)],translation rate constant (ksp) replicate [molecules/(mRNA*h)],translation rate constant (ksp) average [molecules/(mRNA*h)],Half-life,Gene,Cell_type,Method,Species
0,IPI00229543.3;IPI00221463.3;IPI00265761.6;IPI0...,Histone H2A type 1-F;Histone H2A type 3;Histon...,Hist1h2af,Histone H2A type 1-F;Histone H2A type 3;Histon...,Q8CGP5;Q61668;Q8BFU2;A2AB79;P22752;B2RVF0;Q8CG...,NP_783592;NP_835736;NP_783591;NP_835489;NP_835...,NaN,ENSMUST00000073261;ENSMUST00000108817;ENSMUST0...,MGI:2448309;MGI:2448458;MGI:2448306;MGI:244828...,130,...,NaN,NaN,NaN,NaN,NaN,94.71,Hist1h2af,NIH3T3 fibroblasts,Dynamic SILAC,Mouse
1,IPI00229543.3;IPI00221463.3;IPI00265761.6;IPI0...,Histone H2A type 1-F;Histone H2A type 3;Histon...,Hist3h2a,Histone H2A type 1-F;Histone H2A type 3;Histon...,Q8CGP5;Q61668;Q8BFU2;A2AB79;P22752;B2RVF0;Q8CG...,NP_783592;NP_835736;NP_783591;NP_835489;NP_835...,NaN,ENSMUST00000073261;ENSMUST00000108817;ENSMUST0...,MGI:2448309;MGI:2448458;MGI:2448306;MGI:244828...,130,...,NaN,NaN,NaN,NaN,NaN,94.71,Hist3h2a,NIH3T3 fibroblasts,Dynamic SILAC,Mouse
2,IPI00229543.3;IPI00221463.3;IPI00265761.6;IPI0...,Histone H2A type 1-F;Histone H2A type 3;Histon...,Hist1h2ab,Histone H2A type 1-F;Histone H2A type 3;Histon...,Q8CGP5;Q61668;Q8BFU2;A2AB79;P22752;B2RVF0;Q8CG...,NP_783592;NP_835736;NP_783591;NP_835489;NP_835...,NaN,ENSMUST00000073261;ENSMUST00000108817;ENSMUST0...,MGI:2448309;MGI:2448458;MGI:2448306;MGI:244828...,130,...,NaN,NaN,NaN,NaN,NaN,94.71,Hist1h2ab,NIH3T3 fibroblasts,Dynamic SILAC,Mouse
3,IPI00229543.3;IPI00221463.3;IPI00265761.6;IPI0...,Histone H2A type 1-F;Histone H2A type 3;Histon...,Hist1h2ac,Histone H2A type 1-F;Histone H2A type 3;Histon...,Q8CGP5;Q61668;Q8BFU2;A2AB79;P22752;B2RVF0;Q8CG...,NP_783592;NP_835736;NP_783591;NP_835489;NP_835...,NaN,ENSMUST00000073261;ENSMUST00000108817;ENSMUST0...,MGI:2448309;MGI:2448458;MGI:2448306;MGI:244828...,130,...,NaN,NaN,NaN,NaN,NaN,94.71,Hist1h2ac,NIH3T3 fibroblasts,Dynamic SILAC,Mouse
4,IPI00229543.3;IPI00221463.3;IPI00265761.6;IPI0...,Histone H2A type 1-F;Histone H2A type 3;Histon...,Hist1h2ad,Histone H2A type 1-F;Histone H2A type 3;Histon...,Q8CGP5;Q61668;Q8BFU2;A2AB79;P22752;B2RVF0;Q8CG...,NP_783592;NP_835736;NP_783591;NP_835489;NP_835...,NaN,ENSMUST00000073261;ENSMUST00000108817;ENSMUST0...,MGI:2448309;MGI:2448458;MGI:2448306;MGI:244828...,130,...,NaN,NaN,NaN,NaN,NaN,94.71,Hist1h2ad,NIH3T3 fibroblasts,Dynamic SILAC,Mouse


In [ ]:
data20['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data20 = data20[~(pd.isna(data20['Half-life']) | pd.isna(data20['Gene']))]
print(pd.to_numeric(data20['Half-life'], errors='coerce').notna().all())
data20.head()

data20 = data20[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data20.to_csv(save_as, index=False)

True


### Dataset 22

In [60]:
idx = 21
data21 = pd.read_csv(legend1.loc[idx, 'Path'])
print(legend1.loc[idx, 'Species'])
data21.head()

Human


,ProteinGroup.id,UniProt identifier(s),Gene name(s),Protein name(s),Potential contaminant,Number of proteins in group,Unique peptides,Sequence coverage [%],Valid values for K,Cell culture replicate values for K,...,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,3451,Q13685,AAMP,Angio-associated migratory cell protein,NaN,1,13,41.2,8,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4121,Q5JTZ9,AARS2,Alanine--tRNA ligase;mitochondrial,NaN,1,16,22.4,8,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7546,Q9NY61,AATF,Protein AATF,NaN,1,10,25.9,8,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2753,P61221,ABCE1,ATP-binding cassette sub-family E member 1,NaN,1,24,41.1,8,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2234,P42765,ACAA2,3-ketoacyl-CoA thiolase;mitochondrial,NaN,1,18,61.7,8,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Separate rows with multiple genes
data21['Gene name(s)'] = data21['Gene name(s)'].str.split(";")
data21 = data21.explode('Gene name(s)')
data21['Gene name(s)'] = data21['Gene name(s)'].str.strip()
data21 = data21.reset_index(drop=True)

# Standardize columns
data21['Half-life'] = data21['T1/2 [h]']
data21['Gene'] = data21['Gene name(s)']	
data21['Cell_type'] = legend1.loc[idx, 'Cell_type']
data21['Method'] = legend1.loc[idx, 'Method']
data21['Species'] = legend1.loc[idx, 'Species']

data21.head()

,ProteinGroup.id,UniProt identifier(s),Gene name(s),Protein name(s),Potential contaminant,Number of proteins in group,Unique peptides,Sequence coverage [%],Valid values for K,Cell culture replicate values for K,...,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Half-life,Gene,Cell_type,Method,Species
0,3451,Q13685,AAMP,Angio-associated migratory cell protein,NaN,1,13,41.2,8,4,...,NaN,NaN,NaN,NaN,NaN,10.60229913,AAMP,HeLa cells,Pulsed SILAC,Human
1,4121,Q5JTZ9,AARS2,Alanine--tRNA ligase;mitochondrial,NaN,1,16,22.4,8,4,...,NaN,NaN,NaN,NaN,NaN,42.75216253,AARS2,HeLa cells,Pulsed SILAC,Human
2,7546,Q9NY61,AATF,Protein AATF,NaN,1,10,25.9,8,4,...,NaN,NaN,NaN,NaN,NaN,19.27193058,AATF,HeLa cells,Pulsed SILAC,Human
3,2753,P61221,ABCE1,ATP-binding cassette sub-family E member 1,NaN,1,24,41.1,8,4,...,NaN,NaN,NaN,NaN,NaN,69.45857683,ABCE1,HeLa cells,Pulsed SILAC,Human
4,2234,P42765,ACAA2,3-ketoacyl-CoA thiolase;mitochondrial,NaN,1,18,61.7,8,4,...,NaN,NaN,NaN,NaN,NaN,105.66268,ACAA2,HeLa cells,Pulsed SILAC,Human


In [ ]:
data21['Study'] = re.sub("_.*", "", legend1.loc[idx, 'File'])  

# Misc clean up
data21 = data21[~(pd.isna(data21['Half-life']) | pd.isna(data21['Gene']))]
data21['Half-life'] = data21['Half-life'].replace("inf.", np.inf)
print(pd.to_numeric(data21['Half-life'], errors='coerce').notna().all())

data21 = data21[std_columns]
save_as = re.sub(".csv", "", legend1.loc[idx, 'Path']) + "_" + legend1.loc[idx, 'Species'] + "_" + re.sub(" ", "_", legend1.loc[idx, 'Cell_type']) + "_standardized.csv"
data21.to_csv(save_as, index=False)

True
